In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# Set up Chrome driver
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

# Navigate to the website
url = 'https://www.charitiesnys.com/RegistrySearch/search_charities.jsp'
driver.get(url)

# Wait for the pagination links to be present
wait = WebDriverWait(driver, 10)

# Identify the search input and submit button
search_input = driver.find_element(By.XPATH, '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]')
search_input.send_keys('0')
search_button = driver.find_element(By.XPATH, '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]')
search_button.click()

# Wait for the table to load
time.sleep(4)

# Create an empty list to store the data
data = []

# Iterate through the pages
while True:
    # Extract the data from the current page
    table = driver.find_element(By.CSS_SELECTOR, 'table.Bordered')
    for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
        data.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')])

    # Try to find the next page button
    try:
        next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="next"]')))
        next_page_button.click()
    except TimeoutException:
        # If the next page button is not found, break the loop
        break

# Update dataframe with header 
df = pd.DataFrame(data, columns=["Organization Name", "NY Reg #", "EIN", "Registrant Type", "City", "State"])
df.dropna(inplace=True)

# Display the DataFrame
display(df)

# Close the browser
driver.quit()


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
10,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


In [2]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3://module10assignrm/' #specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True) 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

/Users/rujhanmaniktalia/anaconda3/lib/python3.11/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Successfull uploaded file to location:s3://module10assignrm/charities_bureau_scrape_20240414182815.csv
